<a href="https://colab.research.google.com/github/ElenaVillano/sentiment_analysis_tweets/blob/main/notebooks/modelado_ejemplo_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install "git+https://github.com/ElenaVillano/sentiment_analysis_tweets.git#egg=nlptweet&subdirectory=src" --quiet

In [23]:
#!pip uninstall nlptweet

Uninstalling nlptweet-0.1:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/nlp/*
    /usr/local/lib/python3.7/dist-packages/nlptweet-0.1.dist-info/*
Proceed (y/n)? yes
Your response ('yes') was not one of the expected responses: y, n
Proceed (y/n)? y
  Successfully uninstalled nlptweet-0.1


In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import nltk
#import re
import timeit
import string
# Nuestro paquete
import nlp


#call the nltk downloader
nltk.download('punkt')

from dateutil import parser

from sklearn.model_selection import train_test_split

# Carga un set de stopwords predefinidas
from nltk.corpus import stopwords

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Nombramiento de columnas
col_names = ['target', # Polaridad del twet 0=negativo, 2=neutral, 4=positivo
             'ids',    # ID tweet
             'date',   # Fecha y hora del tweet
             'flag',   # QUERY
             'user',   # Usuario del tweet
             'text']   # Texto del tweety

# Carga y limpieza de datos

In [6]:
# Requiered to select a file to be imported into colab
# Not useful if running locally
from google.colab import files
uploaded = files.upload()

Saving testdata_manual_2009_06_14.csv to testdata_manual_2009_06_14 (1).csv


In [5]:
training =  pd.read_csv('smaller_sample_500000.csv',
                 encoding='latin-1')

In [7]:
test = pd.read_csv('testdata_manual_2009_06_14.csv', names=col_names)

In [8]:
print(training.shape)
print(test.shape)

(500000, 6)
(498, 6)


In [9]:
training.head(50)

,target,ids,date,flag,user,text
0,0,1991696611,Mon Jun 01 07:30:14 PDT 2009,NO_QUERY,LeMarianne,I'll be having ER duty tomorrow. This will be...
1,0,2069438971,Sun Jun 07 15:52:53 PDT 2009,NO_QUERY,serpico007,Finished watching the whole season of 24 today...
2,0,1679162446,Sat May 02 08:41:10 PDT 2009,NO_QUERY,katiee1619,wow. i havent been on twitter in awhile.
3,0,2188613884,Mon Jun 15 22:13:26 PDT 2009,NO_QUERY,thegam310,Powered through a 3 hour shift at work today. ...
4,4,2189998539,Tue Jun 16 01:28:54 PDT 2009,NO_QUERY,mattlxs,@JJaeng @tontenn thats good to hear i wanna g...
5,4,2062670089,Sat Jun 06 23:55:55 PDT 2009,NO_QUERY,MsShaleah2u,@ItsaPROBLEM lol that sounds about right! U c...
6,0,1962822149,Fri May 29 11:44:05 PDT 2009,NO_QUERY,tgjerusalem,SHIT I lost my Star Trek ring! Starfleet Ac...
7,4,1969947664,Sat May 30 01:50:35 PDT 2009,NO_QUERY,xLakeEffectKid,@Rogues_Gallery hehe you should definitely go ...
8,0,1964083873,Fri May 29 13:41:59 PDT 2009,NO_QUERY,paigedelperdang,@xoxobb11 you got it all done?? even the labs?...
9,4,2175468855,Mon Jun 15 01:04:22 PDT 2009,NO_QUERY,DazzlingMichele,@DeeDeeB83 Hey Mrs. 'course I found you. I hav...


## Convertimos a minúsculas

In [10]:
from nlp.preprocessing import convierte_a_minusculas

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
training = convierte_a_minusculas(training)
test = convierte_a_minusculas(test)

In [78]:
training.head(50)

,target,ids,date,flag,user,text
0,0,1991696611,mon jun 01 07:30:14 pdt 2009,no_query,lemarianne,i'll be having er duty tomorrow. this will be...
1,0,2069438971,sun jun 07 15:52:53 pdt 2009,no_query,serpico007,finished watching the whole season of 24 today...
2,0,1679162446,sat may 02 08:41:10 pdt 2009,no_query,katiee1619,wow. i havent been on twitter in awhile.
3,0,2188613884,mon jun 15 22:13:26 pdt 2009,no_query,thegam310,powered through a 3 hour shift at work today. ...
4,4,2189998539,tue jun 16 01:28:54 pdt 2009,no_query,mattlxs,@jjaeng @tontenn thats good to hear i wanna g...
5,4,2062670089,sat jun 06 23:55:55 pdt 2009,no_query,msshaleah2u,@itsaproblem lol that sounds about right! u c...
6,0,1962822149,fri may 29 11:44:05 pdt 2009,no_query,tgjerusalem,shit i lost my star trek ring! starfleet ac...
7,4,1969947664,sat may 30 01:50:35 pdt 2009,no_query,xlakeeffectkid,@rogues_gallery hehe you should definitely go ...
8,0,1964083873,fri may 29 13:41:59 pdt 2009,no_query,paigedelperdang,@xoxobb11 you got it all done?? even the labs?...
9,4,2175468855,mon jun 15 01:04:22 pdt 2009,no_query,dazzlingmichele,@deedeeb83 hey mrs. 'course i found you. i hav...


## Reemplazamos las URLs con el texto "URL"

In [12]:
from nlp.preprocessing import reemplazar_urls

In [13]:
training['text'] = training['text'].map(lambda s: reemplazar_urls(s))
test['text'] = test['text'].map(lambda s: reemplazar_urls(s))

In [14]:
training.head(50)

,target,ids,date,flag,user,text
0,0,1991696611,mon jun 01 07:30:14 pdt 2009,no_query,lemarianne,i'll be having er duty tomorrow. this will be...
1,0,2069438971,sun jun 07 15:52:53 pdt 2009,no_query,serpico007,finished watching the whole season of 24 today...
2,0,1679162446,sat may 02 08:41:10 pdt 2009,no_query,katiee1619,wow. i havent been on twitter in awhile.
3,0,2188613884,mon jun 15 22:13:26 pdt 2009,no_query,thegam310,powered through a 3 hour shift at work today. ...
4,4,2189998539,tue jun 16 01:28:54 pdt 2009,no_query,mattlxs,@jjaeng @tontenn thats good to hear i wanna g...
5,4,2062670089,sat jun 06 23:55:55 pdt 2009,no_query,msshaleah2u,@itsaproblem lol that sounds about right! u c...
6,0,1962822149,fri may 29 11:44:05 pdt 2009,no_query,tgjerusalem,shit i lost my star trek ring! starfleet ac...
7,4,1969947664,sat may 30 01:50:35 pdt 2009,no_query,xlakeeffectkid,@rogues_gallery hehe you should definitely go ...
8,0,1964083873,fri may 29 13:41:59 pdt 2009,no_query,paigedelperdang,@xoxobb11 you got it all done?? even the labs?...
9,4,2175468855,mon jun 15 01:04:22 pdt 2009,no_query,dazzlingmichele,@deedeeb83 hey mrs. 'course i found you. i hav...


## Reemplazamos las menciones @ con la palabra "USER_MENTION"



In [33]:
from nlp.preprocessing import reemplazar_usuarios

In [34]:
training['text'] = training['text'].map(lambda s: reemplazar_usuarios(s))
test['text'] = test['text'].map(lambda s: reemplazar_usuarios(s))

In [35]:
training.head(50)

,target,ids,date,flag,user,text
0,0,1991696611,mon jun 01 07:30:14 pdt 2009,no_query,lemarianne,i'll be having er duty tomorrow. this will be...
1,0,2069438971,sun jun 07 15:52:53 pdt 2009,no_query,serpico007,finished watching the whole season of 24 today...
2,0,1679162446,sat may 02 08:41:10 pdt 2009,no_query,katiee1619,wow. i havent been on twitter in awhile.
3,0,2188613884,mon jun 15 22:13:26 pdt 2009,no_query,thegam310,powered through a 3 hour shift at work today. ...
4,4,2189998539,tue jun 16 01:28:54 pdt 2009,no_query,mattlxs,@jjaeng @tontenn thats good to hear i wanna g...
5,4,2062670089,sat jun 06 23:55:55 pdt 2009,no_query,msshaleah2u,@itsaproblem lol that sounds about right! u c...
6,0,1962822149,fri may 29 11:44:05 pdt 2009,no_query,tgjerusalem,shit i lost my star trek ring! starfleet ac...
7,4,1969947664,sat may 30 01:50:35 pdt 2009,no_query,xlakeeffectkid,@rogues_gallery hehe you should definitely go ...
8,0,1964083873,fri may 29 13:41:59 pdt 2009,no_query,paigedelperdang,@xoxobb11 you got it all done?? even the labs?...
9,4,2175468855,mon jun 15 01:04:22 pdt 2009,no_query,dazzlingmichele,@deedeeb83 hey mrs. 'course i found you. i hav...


## Quitamos el Hashtag , pero dejamos la palabra

In [18]:
# The regular expression used to match hashtags is #(\S+).
from nlp.preprocessing import quitar_hashtag

In [19]:
training['text'] = training['text'].map(lambda s: quitar_hashtag(s))
test['text'] = test['text'].map(lambda s: quitar_hashtag(s))

In [20]:
training.head(50)

,target,ids,date,flag,user,text
0,0,1991696611,mon jun 01 07:30:14 pdt 2009,no_query,lemarianne,i'll be having er duty tomorrow. this will be...
1,0,2069438971,sun jun 07 15:52:53 pdt 2009,no_query,serpico007,finished watching the whole season of 24 today...
2,0,1679162446,sat may 02 08:41:10 pdt 2009,no_query,katiee1619,wow. i havent been on twitter in awhile.
3,0,2188613884,mon jun 15 22:13:26 pdt 2009,no_query,thegam310,powered through a 3 hour shift at work today. ...
4,4,2189998539,tue jun 16 01:28:54 pdt 2009,no_query,mattlxs,@jjaeng @tontenn thats good to hear i wanna g...
5,4,2062670089,sat jun 06 23:55:55 pdt 2009,no_query,msshaleah2u,@itsaproblem lol that sounds about right! u c...
6,0,1962822149,fri may 29 11:44:05 pdt 2009,no_query,tgjerusalem,shit i lost my star trek ring! starfleet ac...
7,4,1969947664,sat may 30 01:50:35 pdt 2009,no_query,xlakeeffectkid,@rogues_gallery hehe you should definitely go ...
8,0,1964083873,fri may 29 13:41:59 pdt 2009,no_query,paigedelperdang,@xoxobb11 you got it all done?? even the labs?...
9,4,2175468855,mon jun 15 01:04:22 pdt 2009,no_query,dazzlingmichele,@deedeeb83 hey mrs. 'course i found you. i hav...


## Quitamos los Retweets, sólo la palabra RT , dejamos el comentario

In [21]:
#  The regular expression used to match retweets is \brt\b.
from nlp.preprocessing import quitar_RT

In [22]:
training['text'] = training['text'].map(lambda s: quitar_RT(s))
test['text'] = test['text'].map(lambda s: quitar_RT(s))

In [23]:
training.head(50)

,target,ids,date,flag,user,text
0,0,1991696611,mon jun 01 07:30:14 pdt 2009,no_query,lemarianne,i'll be having er duty tomorrow. this will be...
1,0,2069438971,sun jun 07 15:52:53 pdt 2009,no_query,serpico007,finished watching the whole season of 24 today...
2,0,1679162446,sat may 02 08:41:10 pdt 2009,no_query,katiee1619,wow. i havent been on twitter in awhile.
3,0,2188613884,mon jun 15 22:13:26 pdt 2009,no_query,thegam310,powered through a 3 hour shift at work today. ...
4,4,2189998539,tue jun 16 01:28:54 pdt 2009,no_query,mattlxs,@jjaeng @tontenn thats good to hear i wanna g...
5,4,2062670089,sat jun 06 23:55:55 pdt 2009,no_query,msshaleah2u,@itsaproblem lol that sounds about right! u c...
6,0,1962822149,fri may 29 11:44:05 pdt 2009,no_query,tgjerusalem,shit i lost my star trek ring! starfleet ac...
7,4,1969947664,sat may 30 01:50:35 pdt 2009,no_query,xlakeeffectkid,@rogues_gallery hehe you should definitely go ...
8,0,1964083873,fri may 29 13:41:59 pdt 2009,no_query,paigedelperdang,@xoxobb11 you got it all done?? even the labs?...
9,4,2175468855,mon jun 15 01:04:22 pdt 2009,no_query,dazzlingmichele,@deedeeb83 hey mrs. 'course i found you. i hav...


## Quitamos espacios o puntos extras

In [24]:
## Reemplazamos los dobles puntos (o más) con un espacio , dos o más espacios con 1 espacio y hacemos strip de espacios y comillas
# Strip any punctuation [’"?!,.():;-'] from the word ??

from nlp.preprocessing import quitar_caracteres_especiales

In [99]:
training['text'] = training['text'].map(lambda s: quitar_caracteres_especiales(s))
test['text'] = test['text'].map(lambda s: quitar_caracteres_especiales(s))

In [101]:
training.head(50)

,target,ids,date,flag,user,text
0,0,1991696611,mon jun 01 07:30:14 pdt 2009,no_query,lemarianne,i'll be having er duty tomorrow this will be f...
1,0,2069438971,sun jun 07 15:52:53 pdt 2009,no_query,serpico007,finished watching the whole season of 24 today...
2,0,1679162446,sat may 02 08:41:10 pdt 2009,no_query,katiee1619,wow i havent been on twitter in awhile
3,0,2188613884,mon jun 15 22:13:26 pdt 2009,no_query,thegam310,powered through a 3 hour shift at work today f...
4,4,2189998539,tue jun 16 01:28:54 pdt 2009,no_query,mattlxs,user_mention user_mention thats good to hear i...
5,4,2062670089,sat jun 06 23:55:55 pdt 2009,no_query,msshaleah2u,user_mention lol that sounds about right u can...
6,0,1962822149,fri may 29 11:44:05 pdt 2009,no_query,tgjerusalem,shit i lost my star trek ring starfleet academ...
7,4,1969947664,sat may 30 01:50:35 pdt 2009,no_query,xlakeeffectkid,user_mention hehe you should definitely go see...
8,0,1964083873,fri may 29 13:41:59 pdt 2009,no_query,paigedelperdang,user_mention you got it all done even the labs...
9,4,2175468855,mon jun 15 01:04:22 pdt 2009,no_query,dazzlingmichele,user_mention hey mrs 'course i found you i hav...


## Quitamos expresiones con letras repetidas

In [102]:
#Convert 2 or more letter repetitions to 2 letters.
#Some people send tweets like I am sooooo
#happpppy adding multiple characters to emphasize
#on certain words. This is done to handle such tweets
#by converting them to I am soo happy
from nlp.preprocessing import quitar_letras_repetidas

In [103]:
training['text'] = training['text'].map(lambda s: quitar_letras_repetidas(s))
test['text'] = test['text'].map(lambda s: quitar_letras_repetidas(s))

In [104]:
training.head(50)

,target,ids,date,flag,user,text
0,0,1991696611,mon jun 01 07:30:14 pdt 2009,no_query,lemarianne,i'll be having er duty tomorrow this will be f...
1,0,2069438971,sun jun 07 15:52:53 pdt 2009,no_query,serpico007,finished watching the whole season of 24 today...
2,0,1679162446,sat may 02 08:41:10 pdt 2009,no_query,katiee1619,wow i havent been on twitter in awhile
3,0,2188613884,mon jun 15 22:13:26 pdt 2009,no_query,thegam310,powered through a 3 hour shift at work today f...
4,4,2189998539,tue jun 16 01:28:54 pdt 2009,no_query,mattlxs,user_mention user_mention thats good to hear i...
5,4,2062670089,sat jun 06 23:55:55 pdt 2009,no_query,msshaleah2u,user_mention lol that sounds about right u can...
6,0,1962822149,fri may 29 11:44:05 pdt 2009,no_query,tgjerusalem,shit i lost my star trek ring starfleet academ...
7,4,1969947664,sat may 30 01:50:35 pdt 2009,no_query,xlakeeffectkid,user_mention hehe you should definitely go see...
8,0,1964083873,fri may 29 13:41:59 pdt 2009,no_query,paigedelperdang,user_mention you got it all done even the labs...
9,4,2175468855,mon jun 15 01:04:22 pdt 2009,no_query,dazzlingmichele,user_mention hey mrs 'course i found you i hav...


## Quitar caracteres nonascii

In [105]:
from nlp.preprocessing import quitar_nonascii

In [106]:
training['text'] = training['text'].map(lambda s: quitar_nonascii(s))
test['text'] = test['text'].map(lambda s: quitar_nonascii(s))

## Removemos stopwords

In [107]:
nltk.download('stopwords') # Use predefined list of stopwords from nltk

STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [112]:
# Aux funcción para substituir abreviaciones
def decontracted(phrase):
  phrase = re.sub(r"won't", "will not", phrase)
  phrase = re.sub(r"can\'t", "can not", phrase)
  phrase = re.sub(r"n\'t", " not", phrase)
  phrase = re.sub(r"\'re", " are", phrase)
  phrase = re.sub(r"\'s", " is", phrase)
  phrase = re.sub(r"\'d", " would", phrase)
  phrase = re.sub(r"\'ll", " will", phrase)
  phrase = re.sub(r"\'t", " not", phrase)
  phrase = re.sub(r"\'ve", " have", phrase)
  phrase = re.sub(r"\'m", " am", phrase)
  return phrase

In [113]:
training['text'] = training['text'].map(lambda s: decontracted(s))
test['text'] = test['text'].map(lambda s: decontracted(s))

In [110]:
training['text'] = training['text'].map(lambda s: remove_stopwords(s))
test['atext'] = test['text'].map(lambda s: remove_stopwords(s))

In [114]:
training

,target,ids,date,flag,user,text
0,0,1991696611,mon jun 01 07:30:14 pdt 2009,no_query,lemarianne,i will er duty tomorrow fun exciting i will pr...
1,0,2069438971,sun jun 07 15:52:53 pdt 2009,no_query,serpico007,finished watching whole season 24 today bad th...
2,0,1679162446,sat may 02 08:41:10 pdt 2009,no_query,katiee1619,wow havent twitter awhile
3,0,2188613884,mon jun 15 22:13:26 pdt 2009,no_query,thegam310,powered 3 hour shift work today felt real sick...
4,4,2189998539,tue jun 16 01:28:54 pdt 2009,no_query,mattlxs,user_mention user_mention thats good hear wann...
...,...,...,...,...,...,...
499995,0,1564486206,mon apr 20 03:43:10 pdt 2009,no_query,meggie1564,mornings horrible
499996,4,1822394689,sat may 16 19:28:44 pdt 2009,no_query,miizronnie,user_mention thats good im alright thnx yeahh ...
499997,0,1970707319,sat may 30 04:55:29 pdt 2009,no_query,chrismogee,office saturday hopefully getting early get ea...
499998,4,2055591155,sat jun 06 09:35:48 pdt 2009,no_query,articlescholar,eating froot loops url


## Stemming con NLTK

In [ ]:
#create an object of class PorterStemmer
porter = PorterStemmer()
lancaster=LancasterStemmer()

def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [ ]:
training['text'] = training['text'].map(lambda s: stemSentence(s))
test['atext'] = test['text'].map(lambda s: stemSentence(s))

In [ ]:
training

,target,ids,date,flag,user,text
0,0,1467810369,mon apr 06 22:19:45 pdt 2009,no_query,_thespecialone_,user_ment url aww bummer shoulda got david car...
1,0,1467810672,mon apr 06 22:19:49 pdt 2009,no_query,scotthamilton,upset updat facebook text might cri result sch...
2,0,1467810917,mon apr 06 22:19:53 pdt 2009,no_query,mattycus,user_ment dive mani time ball manag save 50 % ...
3,0,1467811184,mon apr 06 22:19:57 pdt 2009,no_query,ellectf,whole bodi feel itchi like fire
4,0,1467811193,mon apr 06 22:19:57 pdt 2009,no_query,karoli,user_ment behav mad see
...,...,...,...,...,...,...
1599995,4,2193601966,tue jun 16 08:40:49 pdt 2009,no_query,amandamarie1028,woke school best feel ever
1599996,4,2193601969,tue jun 16 08:40:49 pdt 2009,no_query,thewdboards,thewdbcom cool hear old walt interview url
1599997,4,2193601991,tue jun 16 08:40:49 pdt 2009,no_query,bpbabe,readi mojo makeov ask detail
1599998,4,2193602064,tue jun 16 08:40:49 pdt 2009,no_query,tinydiamondz,happi 38th birthday boo time tupac amaru shakur


Quitamos datos al entrenamiento, empecemos con 100,000 

In [ ]:
data_set_1 = training= training.head(100000)

In [ ]:
data_set_1

,target,ids,date,flag,user,text
0,0,1467810369,mon apr 06 22:19:45 pdt 2009,no_query,_thespecialone_,user_ment url aww bummer shoulda got david car...
1,0,1467810672,mon apr 06 22:19:49 pdt 2009,no_query,scotthamilton,upset updat facebook text might cri result sch...
2,0,1467810917,mon apr 06 22:19:53 pdt 2009,no_query,mattycus,user_ment dive mani time ball manag save 50 % ...
3,0,1467811184,mon apr 06 22:19:57 pdt 2009,no_query,ellectf,whole bodi feel itchi like fire
4,0,1467811193,mon apr 06 22:19:57 pdt 2009,no_query,karoli,user_ment behav mad see
...,...,...,...,...,...,...
99995,0,1793820630,thu may 14 04:05:58 pdt 2009,no_query,babybigmouth,son develop new habit wake 530am second pot co...
99996,0,1793820812,thu may 14 04:06:00 pdt 2009,no_query,paramoreroxx,look like router broke tweet fone
99997,0,1793820858,thu may 14 04:06:00 pdt 2009,no_query,beckypope,realli dont want colleg right wish sunni
99998,0,1793820982,thu may 14 04:06:02 pdt 2009,no_query,xmissxxxmurderx,user_ment *offer pepto*


# Entrenamos un Tokenizer

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers

# Entrena un Tokenizer. Consiste en:
# Crea un diccionario numerado de las palabras existentes en el corpus, y devuelve
# por cada palabra, el número entero de su índice en el diccionario.
# En este caso, considera las N palabras más frecuentes.
# oov_token = constante asiganda para palabras fuera del vocabulario (NOT USED HERE)


max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data_set_1.text)
sequences = tokenizer.texts_to_sequences(data_set_1.text)
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)

[[   0    0    0 ...  555 1577    6]
 [   0    0    0 ...   10  217  706]
 [   0    0    0 ...  357    3 2676]
 ...
 [   0    0    0 ...   60   20  369]
 [   0    0    0 ...    2    1 1207]
 [   0    0    0 ...   37   87  313]]


## Separamos en train, test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tweets, data_set_1.target.values, test_size=0.3, random_state=23042021)

In [ ]:
print("X_train", X_train.shape)
print("y_train", y_train.shape)
print("X_test", X_test.shape)
print("y_test", y_test.shape)

X_train (70000, 200)
y_train (70000,)
X_test (30000, 200)
y_test (30000,)


# Creamos el modelo

### RNN, tomado de: https://www.kaggle.com/smitshah00/sentiment-analysis-ml-rnn

In [ ]:
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint


model2 = Sequential()
model2.add(layers.Embedding(max_words, 128))
model2.add(layers.LSTM(64,dropout=0.5))
model2.add(layers.Dense(16, activation='relu'))
model2.add(layers.Dense(8, activation='relu'))
model2.add(layers.Dense(1,activation='sigmoid'))
model2.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         640000    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 16)                1040      
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 690,593
Trainable params: 690,593
Non-trainable params: 0
_________________________________________________________________


In [ ]:
checkpoint2 = ModelCheckpoint("rnn_model.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model2.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test),callbacks=[checkpoint2])

Epoch 1/10
2188/2188 [==============================] - 366s 166ms/step - loss: 0.0219 - accuracy: 0.9963 - val_loss: 2.3566e-06 - val_accuracy: 1.0000

Epoch 00001: val_accuracy improved from -inf to 1.00000, saving model to rnn_model.hdf5
Epoch 2/10
2188/2188 [==============================] - 360s 165ms/step - loss: 1.5734e-06 - accuracy: 1.0000 - val_loss: 4.3041e-07 - val_accuracy: 1.0000

Epoch 00002: val_accuracy did not improve from 1.00000
Epoch 3/10
2188/2188 [==============================] - 361s 165ms/step - loss: 3.1633e-07 - accuracy: 1.0000 - val_loss: 1.1139e-07 - val_accuracy: 1.0000

Epoch 00003: val_accuracy did not improve from 1.00000
Epoch 4/10
2188/2188 [==============================] - 360s 165ms/step - loss: 8.4086e-08 - accuracy: 1.0000 - val_loss: 3.1955e-08 - val_accuracy: 1.0000

Epoch 00004: val_accuracy did not improve from 1.00000
Epoch 5/10
2188/2188 [==============================] - 371s 170ms/step - loss: 2.4389e-08 - accuracy: 1.0000 - val_loss: 9

# Probamos

In [ ]:
model = keras.models.load_model('rnn_model.hdf5')
sequence = tokenizer.texts_to_sequences(['this data science article is the best ever'])
test = pad_sequences(sequence, maxlen=max_len)
pred = model.predict(test)
if pred > 0.5:
  print('Positive')
else:
  print('Negative')